In [3]:
### Load libraries
import os
import sem.es as es
import numpy as np
import pandas as pd
import datetime

# Prepare data and metadata / annotation

In [9]:
# Data
file_data = "/scratch/data-for_fast_access/pub-others/tabula_muris_180920/tabula_muris.umi.csv.gz"
df_data = pd.read_csv(file_data) # this takes 12-14 min for tabula_muris! (Pandas is slow!)
# Metadata
file_metadata = "/scratch/data-for_fast_access/pub-others/tabula_muris_180920/tabula_muris.metadata.csv"
df_metadata = pd.read_csv(file_metadata)

In [13]:
df_metadata.head()

,cell_id,nGene,nReads,tissue,subtissue_clean,celltype,tissue_celltype,tissue_subtissue_celltype
0,A1.B000126.3_39_F.1.1,3125,599257,Skin,Telogen,epidermal_cell,Skin.epidermal_cell,Skin.Telogen.epidermal_cell
1,A1.B003283.3_38_F.1.1,5543,2585048,Skin,Telogen,epidermal_cell,Skin.epidermal_cell,Skin.Telogen.epidermal_cell
2,A1.MAA000435.3_10_M.1.1,5023,1748535,Skin,Anagen,basal_cell_of_epidermis,Skin.basal_cell_of_epidermis,Skin.Anagen.basal_cell_of_epidermis
3,A1.MAA000549.3_8_M.1.1,3846,309793,Skin,Anagen,epidermal_cell,Skin.epidermal_cell,Skin.Anagen.epidermal_cell
4,A1.MAA000614.3_10_M.1.1,3249,1044110,Skin,Telogen,basal_cell_of_epidermis,Skin.basal_cell_of_epidermis,Skin.Telogen.basal_cell_of_epidermis


In [10]:
df_counts = df_data.set_index('gene')
df_counts.head(10)

,A1.B000126.3_39_F.1.1,A1.B003283.3_38_F.1.1,A1.MAA000435.3_10_M.1.1,A1.MAA000549.3_8_M.1.1,A1.MAA000614.3_10_M.1.1,A1.MAA000938.3_8_M.1.1,A10.B003283.3_38_F.1.1,A11.B000126.3_39_F.1.1,A12.B000126.3_39_F.1.1,A12.B003283.3_38_F.1.1,...,O6.MAA001847.3_39_F.1.1,O7.MAA001847.3_39_F.1.1,P2.MAA001847.3_39_F.1.1,P3.MAA000839.3_11_M.1.1,P4.MAA000526.3_9_M.1.1,P4.MAA000839.3_11_M.1.1,P5.MAA000526.3_9_M.1.1,P5.MAA001847.3_39_F.1.1,P6.MAA001847.3_39_F.1.1,P9.MAA001847.3_39_F.1.1
gene,,,,,,,,,,,,,,,,,,,,,
0610005C13Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610007C21Rik,265,1026,0,48,26,87,36,245,0,245,...,74,0,164,0,51,315,1,0,193,0
0610007L01Rik,1,35,4,0,186,2,0,0,0,0,...,72,4,7,0,0,0,0,15,0,33
0610007N19Rik,0,0,0,0,0,0,0,0,0,0,...,11,0,0,0,0,3,0,1,0,0
0610007P08Rik,0,0,0,10,0,0,110,0,0,0,...,0,0,0,0,0,0,0,0,53,0
0610007P14Rik,30,0,24,0,0,0,0,2,0,76,...,0,0,51,0,72,0,0,31,8,0
0610007P22Rik,0,247,0,0,165,0,0,0,0,0,...,0,37,0,0,3,0,0,0,0,0
0610008F07Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009B14Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Mapping MGI to Ensembl gene names

In [11]:
gene_name_mapping = pd.read_csv("/projects/timshel/sc-genetics/sc-genetics/data/gene_annotations/Mus_musculus.GRCm38.90.gene_name_version2ensembl.txt.gz", sep=None, engine='python')
# Converting MGI symbols to lowercase to avoid issues with case sensitivity
gene_name_mapping.gene_name_optimal = gene_name_mapping.gene_name_optimal.map(lambda x:x.lower())
df_counts.index = df_counts.index.map(lambda x:x.lower())
# How many of the dataset's genes are in the mapping list and how many aren't
in_mapping_bool = df_counts.index.isin(gene_name_mapping.gene_name_optimal)
print('How many genes can be mapped from MGI name to Ensembl mouse names?')
print(pd.Series(in_mapping_bool).value_counts())

How many genes can be mapped from MGI name to Ensembl mouse names?
True     20874
False     2467
dtype: int64


In [12]:
mapping_dict = pd.Series(gene_name_mapping.ensembl_gene_id.values,index=gene_name_mapping.gene_name_optimal).to_dict() # Quickest way according to https://stackoverflow.com/questions/17426292/
df_counts.index = df_counts.index.map(mapping_dict)
df_counts.head(10)

,A1.B000126.3_39_F.1.1,A1.B003283.3_38_F.1.1,A1.MAA000435.3_10_M.1.1,A1.MAA000549.3_8_M.1.1,A1.MAA000614.3_10_M.1.1,A1.MAA000938.3_8_M.1.1,A10.B003283.3_38_F.1.1,A11.B000126.3_39_F.1.1,A12.B000126.3_39_F.1.1,A12.B003283.3_38_F.1.1,...,O6.MAA001847.3_39_F.1.1,O7.MAA001847.3_39_F.1.1,P2.MAA001847.3_39_F.1.1,P3.MAA000839.3_11_M.1.1,P4.MAA000526.3_9_M.1.1,P4.MAA000839.3_11_M.1.1,P5.MAA000526.3_9_M.1.1,P5.MAA001847.3_39_F.1.1,P6.MAA001847.3_39_F.1.1,P9.MAA001847.3_39_F.1.1
gene,,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000109644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NaN,265,1026,0,48,26,87,36,245,0,245,...,74,0,164,0,51,315,1,0,193,0
NaN,1,35,4,0,186,2,0,0,0,0,...,72,4,7,0,0,0,0,15,0,33
NaN,0,0,0,0,0,0,0,0,0,0,...,11,0,0,0,0,3,0,1,0,0
NaN,0,0,0,10,0,0,110,0,0,0,...,0,0,0,0,0,0,0,0,53,0
NaN,30,0,24,0,0,0,0,2,0,76,...,0,0,51,0,72,0,0,31,8,0
NaN,0,247,0,0,165,0,0,0,0,0,...,0,37,0,0,3,0,0,0,0,0
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
### Inspect metadata. Note that there are 48 unique cell-types.
# N.B. the current implementation only accepts 1 by n_cells numpy arrays for the annotation.
# We will have to match the contents of the metadata file to the cell-id's of the counts dataframe

print(df_metadata.shape)
n_cell_types = df_metadata["tissue_celltype"].astype(str)
print(np.unique(n_cell_types).size)
df_metadata.head()

(44949, 8)
115


,cell_id,nGene,nReads,tissue,subtissue_clean,celltype,tissue_celltype,tissue_subtissue_celltype
0,A1.B000126.3_39_F.1.1,3125,599257,Skin,Telogen,epidermal_cell,Skin.epidermal_cell,Skin.Telogen.epidermal_cell
1,A1.B003283.3_38_F.1.1,5543,2585048,Skin,Telogen,epidermal_cell,Skin.epidermal_cell,Skin.Telogen.epidermal_cell
2,A1.MAA000435.3_10_M.1.1,5023,1748535,Skin,Anagen,basal_cell_of_epidermis,Skin.basal_cell_of_epidermis,Skin.Anagen.basal_cell_of_epidermis
3,A1.MAA000549.3_8_M.1.1,3846,309793,Skin,Anagen,epidermal_cell,Skin.epidermal_cell,Skin.Anagen.epidermal_cell
4,A1.MAA000614.3_10_M.1.1,3249,1044110,Skin,Telogen,basal_cell_of_epidermis,Skin.basal_cell_of_epidermis,Skin.Telogen.basal_cell_of_epidermis


In [15]:
# Map the cell_type to the cell_id of the raw counts dataframe
# N.B. nan values are propagated!
anno = df_counts.columns.map(df_metadata["tissue_celltype"], na_action="ignore").values

# # Convert to string to ensure uniform type of values. Otherwise numpy will get mad.
anno_str = anno.astype(str)

# Compute Expression Specificity

In [17]:
## Create machine
#N.B. default args for es.object.Machine(preprocess=True)
print("Creating Machine ...")
print("    ", datetime.datetime.now().time())
machine = es.object.Machine(df_counts)
print("    ", datetime.datetime.now().time())
print("    ", machine.df.shape)
machine.df.head()

Creating Machine ...
     15:09:59.244794
     15:12:10.681946
     (22951, 44949)


,A1.B000126.3_39_F.1.1,A1.B003283.3_38_F.1.1,A1.MAA000435.3_10_M.1.1,A1.MAA000549.3_8_M.1.1,A1.MAA000614.3_10_M.1.1,A1.MAA000938.3_8_M.1.1,A10.B003283.3_38_F.1.1,A11.B000126.3_39_F.1.1,A12.B000126.3_39_F.1.1,A12.B003283.3_38_F.1.1,...,O6.MAA001847.3_39_F.1.1,O7.MAA001847.3_39_F.1.1,P2.MAA001847.3_39_F.1.1,P3.MAA000839.3_11_M.1.1,P4.MAA000526.3_9_M.1.1,P4.MAA000839.3_11_M.1.1,P5.MAA000526.3_9_M.1.1,P5.MAA001847.3_39_F.1.1,P6.MAA001847.3_39_F.1.1,P9.MAA001847.3_39_F.1.1
gene,,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000109644,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NaN,1.690491,1.603214,0.000000,0.935867,0.222356,1.365487,0.260079,1.869196,0.0,1.172834,...,0.850037,0.000000,1.409904,0.0,0.850122,1.517171,0.114043,0.000000,1.450732,0.000000
NaN,0.016550,0.126980,0.022619,0.000000,1.022962,0.064918,0.000000,0.000000,0.0,0.000000,...,0.834440,0.082718,0.124099,0.0,0.000000,0.000000,0.000000,0.237875,0.000000,0.441302
NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.181613,0.000000,0.000000,0.0,0.000000,0.033336,0.000000,0.017745,0.000000,0.000000
NaN,0.000000,0.000000,0.000000,0.279748,0.000000,0.000000,0.645847,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.640245,0.000000


In [18]:
### Add annotation
# N.B. default args for add_annotation(run_anova=True, map_genes=True, verbose=False)
# We must set map_genes to false, since the implementation is hard-coded to use a specific gene list

name_anno = "cell_type"

print("Adding Annotation ...")
print("    ", datetime.datetime.now().time())
machine.add_annotation(name_anno, anno_str, map_genes=True, verbose=True)
print("    ", datetime.datetime.now().time())
print("    ", machine.df.shape)

Adding Annotation ...
     15:12:10.714469
Mapping gene id's to ortholog gene id's ...
/nfsdata/projects/alegbe/sc-genetics/src/benchmark
Removed 8203 unmapped genes ...
0 pct unmapped genes
     15:15:53.622777
     (14302, 44949)


In [19]:
### Compute ESw, ESw* and ESmu
# N.B. the current implementation will run all ES Metrics, unless we specify which ones we want.
# To avoid any side-effects, we specify the ES metrics to those used in BMI brain.
# tl;dr: I recommend specifying esms.

# N.B. default args for compute(self, annotations: list=None, esms: list={"ges", "si", "ss", "tstat", "zstw"}, 
#                                verbose: bool=False, compute_meta: bool=False)
# tl;dr: set compute_meta=True.

esm_list = ["ges", "si", "ss", "tstat"]

print("Computing ESws ...")
print("    ", datetime.datetime.now().time())
machine.compute(annotations=[name_anno], esms={*esm_list}, compute_meta=True, verbose=True)
print("    ", datetime.datetime.now().time())

Computing ESws ...
     15:15:53.635295
Computing GES ...
Computing FDR ...
Computing esw_s ...
Computing TSTAT ...
Computing FDR ...
Computing esw_s ...
Computing SI ...
Computing FDR ...
Computing esw_s ...
Computing SS ...
Computing FDR ...
Computing esw_s ...
Computing esw_mu ...
Computed ['cell_type.ges.esw', 'cell_type.ges.esw_null', 'cell_type.ges.pvals', 'cell_type.ges.qvals', 'cell_type.ges.esw_s', 'cell_type.tstat.esw', 'cell_type.tstat.esw_null', 'cell_type.tstat.pvals', 'cell_type.tstat.qvals', 'cell_type.tstat.esw_s', 'cell_type.si.esw', 'cell_type.si.esw_null', 'cell_type.si.pvals', 'cell_type.si.qvals', 'cell_type.si.esw_s', 'cell_type.ss.esw', 'cell_type.ss.esw_null', 'cell_type.ss.pvals', 'cell_type.ss.qvals', 'cell_type.ss.esw_s', 'cell_type.esmu'] ...
     15:17:48.397628


# Inspect and save results

In [20]:
### Do the results match our expectations?
# 48 unique cell-types (including propagated "nan"-cell-type).
machine.metrics['cell_type.esmu'].head()

,Bladder.bladder_cell,Bladder.bladder_urothelial_cell,Brain_Myeloid.macrophage,Brain_Myeloid.microglial_cell,Brain_Non-Myeloid.Bergmann_glial_cell,Brain_Non-Myeloid.astrocyte,Brain_Non-Myeloid.brain_pericyte,Brain_Non-Myeloid.endothelial_cell,Brain_Non-Myeloid.neuron,Brain_Non-Myeloid.oligodendrocyte,...,Spleen.macrophage,Thymus.DN1_thymic_pro-T_cell,Thymus.immature_T_cell,Thymus.leukocyte,Tongue.basal_cell_of_epidermis,Tongue.keratinocyte,Trachea.blood_cell,Trachea.endothelial_cell,Trachea.epithelial_cell,Trachea.mesenchymal_cell
gene,,,,,,,,,,,,,,,,,,,,,
ENSG00000081791,0.02842,0.000000,0.000000,0.043131,0.000000,0.021647,0.147595,0.377622,0.039707,0.240040,...,0.0,0.0,0.000000,0.0,0.331036,0.000000,0.0,0.000000,0.0,0.000000
ENSG00000162929,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.407688,0.219948,...,0.0,0.0,0.000000,0.0,0.038015,0.000000,0.0,0.049549,0.0,0.213655
ENSG00000168887,0.00000,0.000000,0.000000,0.056487,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.192736,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000
ENSG00000162384,0.34127,0.175932,0.000000,0.000000,0.000000,0.000000,0.000000,0.252456,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.060192,0.104673,0.0,0.000000,0.0,0.110991
ENSG00000154274,0.00000,0.679072,0.422476,0.900619,0.359992,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000


In [7]:
### Save everything in machine.metrics
# machine.metrics is a dictionary that holds the results.
# results include esw, esw_null, pvals, qvals and ESmu.
print("Saving results to disk ...")
name_anno = 'relevance'
current_date = "190722"
bench_date = 'benchmark' + current_date


dir_path = "out_{}".format(name_anno)
os.makedirs(dir_path, exist_ok=True) # make dir if it doesn't already exist

### Save results
for m, df in machine.metrics.items():
    fp = "out_{}/{}.{}.mapped.csv.gz".format(name_anno, m, current_date)
    print(fp)
    df.to_csv(fp, compression="gzip")

Saving results to disk ...


# Making benchmark multigenesets for CELLECT

In [5]:
name_of_dataset = 'tabula_muris'

# Finucane top 10% of the tstat significant

In [24]:
es_metric = 'HF-tstat10'
binary_or_cont = 'binary'

tstat_df = machine.metrics["cell_type.tstat.esw"].copy()
tstat_df.where(cond=tstat_df.quantile(0.9)<tstat_df,other=0, inplace=True)
tstat_df.where(cond=tstat_df==0,other=1, inplace=True,)
tstat_df.index.rename(name='gene',inplace=True)
tstat_df.reset_index(inplace=True)
tstat_long_df = pd.melt(tstat_df,id_vars=['gene'],var_name='annotation', value_name='specificity')
multi_geneset_celltypes = tstat_long_df[['annotation','gene','specificity']]
multi_geneset_celltypes = multi_geneset_celltypes.loc[multi_geneset_celltypes.specificity>0]
multi_geneset_celltypes.to_csv('../../data/benchmark_multigenesets/multi_geneset.{}.{}.{}.{}.txt'.format(name_of_dataset, es_metric, binary_or_cont, bench_date),header=None, index=False,sep='\t')

In [25]:
print(len(np.unique(multi_geneset_celltypes.gene)))
multi_geneset_celltypes.head()

14262


,annotation,gene,specificity
3,Bladder.bladder_cell,ENSG00000162384,1.0
56,Bladder.bladder_cell,ENSG00000116667,1.0
76,Bladder.bladder_cell,ENSG00000182831,1.0
79,Bladder.bladder_cell,ENSG00000138944,1.0
82,Bladder.bladder_cell,ENSG00000171067,1.0


# Skene specificity (continuous and binary)

In [26]:
es_metric = 'SkeneSpecificity'
binary_or_cont = 'continuous'

ss_df = machine.metrics["cell_type.ss.esw"].copy()
# ss_df.where(cond=ss_df==0,other=1, inplace=True)
ss_df.index.rename(name='gene',inplace=True)
ss_df.reset_index(inplace=True)
ss_long_df = pd.melt(ss_df,id_vars=['gene'],var_name='annotation', value_name='specificity')
multi_geneset_celltypes = ss_long_df[['annotation','gene','specificity']]
multi_geneset_celltypes = multi_geneset_celltypes.loc[multi_geneset_celltypes.specificity>0]
multi_geneset_celltypes.to_csv('../../data/benchmark_multigenesets/multi_geneset.{}.{}.{}.{}.txt'.format(name_of_dataset, es_metric, binary_or_cont, bench_date),header=None, index=False,sep='\t')

In [27]:
print(len(np.unique(multi_geneset_celltypes.gene)))
multi_geneset_celltypes.head()

14302


,annotation,gene,specificity
0,Bladder.bladder_cell,ENSG00000081791,0.008463
1,Bladder.bladder_cell,ENSG00000162929,0.007354
2,Bladder.bladder_cell,ENSG00000168887,0.003588
3,Bladder.bladder_cell,ENSG00000162384,0.014594
4,Bladder.bladder_cell,ENSG00000154274,0.000032


In [28]:
es_metric = 'SkeneSpecificity'
binary_or_cont = 'binary'

ss_df = machine.metrics["cell_type.ss.esw"].copy()
ss_df.where(cond=ss_df==0,other=1, inplace=True)
ss_df.index.rename(name='gene',inplace=True)
ss_df.reset_index(inplace=True)
ss_long_df = pd.melt(ss_df,id_vars=['gene'],var_name='annotation', value_name='specificity')
multi_geneset_celltypes = ss_long_df[['annotation','gene','specificity']]
multi_geneset_celltypes = multi_geneset_celltypes.loc[multi_geneset_celltypes.specificity>0]
multi_geneset_celltypes.to_csv('../../data/benchmark_multigenesets/multi_geneset.{}.{}.{}.{}.txt'.format(name_of_dataset, es_metric, binary_or_cont, bench_date),header=None, index=False,sep='\t')

In [29]:
print(len(np.unique(multi_geneset_celltypes.gene)))
multi_geneset_celltypes.head()

14302


,annotation,gene,specificity
0,Bladder.bladder_cell,ENSG00000081791,1.0
1,Bladder.bladder_cell,ENSG00000162929,1.0
2,Bladder.bladder_cell,ENSG00000168887,1.0
3,Bladder.bladder_cell,ENSG00000162384,1.0
4,Bladder.bladder_cell,ENSG00000154274,1.0


# Shuffling the es_df to generate null 1

In [30]:
# Example
df = pd.DataFrame({'a':[1,2,3,4,5,6], 'b':[1,2,3,4,5,6]})
print(df)
shuffled_df = df.apply(lambda x: x.sample(frac=1).values)
print(shuffled_df)

   a  b
0  1  1
1  2  2
2  3  3
3  4  4
4  5  5
5  6  6
   a  b
0  4  2
1  3  3
2  2  5
3  5  6
4  6  4
5  1  1


In [31]:
es_metric = 'ESmu-NULL1'
binary_or_cont = 'continuous'
es_df = machine.metrics["cell_type.esmu"].copy()
# es_df.where(cond=es_df==0,other=1, inplace=True)
es_df.index.rename(name='gene',inplace=True)
print(es_df.head())
for i in range(5):
    shuffled_es_df = es_df.apply(lambda x: x.sample(frac=1).values)
    shuffled_es_df.reset_index(inplace=True)
    shuffled_es_long_df = pd.melt(shuffled_es_df,id_vars=['gene'],var_name='annotation', value_name='specificity')
    multi_geneset_celltypes = shuffled_es_long_df[['annotation','gene','specificity']]
    multi_geneset_celltypes = multi_geneset_celltypes.loc[multi_geneset_celltypes.specificity>0]
    multi_geneset_celltypes.to_csv('../../data/benchmark_multigenesets/multi_geneset.{}.{}.{}.{}.{}.txt'.format(name_of_dataset, es_metric, binary_or_cont, bench_date, i),header=None, index=False,sep='\t')
print(shuffled_es_df.head())

                 Bladder.bladder_cell  Bladder.bladder_urothelial_cell  \
gene                                                                     
ENSG00000081791               0.02842                         0.000000   
ENSG00000162929               0.00000                         0.000000   
ENSG00000168887               0.00000                         0.000000   
ENSG00000162384               0.34127                         0.175932   
ENSG00000154274               0.00000                         0.679072   

                 Brain_Myeloid.macrophage  Brain_Myeloid.microglial_cell  \
gene                                                                       
ENSG00000081791                  0.000000                       0.043131   
ENSG00000162929                  0.000000                       0.000000   
ENSG00000168887                  0.000000                       0.056487   
ENSG00000162384                  0.000000                       0.000000   
ENSG00000154274          

In [32]:
es_metric = 'ESmu-NULL1'
binary_or_cont = 'binary'
es_df = machine.metrics["cell_type.esmu"].copy()
es_df.where(cond=es_df==0,other=1, inplace=True)
es_df.index.rename(name='gene',inplace=True)
print(es_df.head())
for i in range(5):
    shuffled_es_df = es_df.apply(lambda x: x.sample(frac=1).values)
    shuffled_es_df.reset_index(inplace=True)
    shuffled_es_long_df = pd.melt(shuffled_es_df,id_vars=['gene'],var_name='annotation', value_name='specificity')
    multi_geneset_celltypes = shuffled_es_long_df[['annotation','gene','specificity']]
    multi_geneset_celltypes = multi_geneset_celltypes.loc[multi_geneset_celltypes.specificity>0]
    multi_geneset_celltypes.to_csv('../../data/benchmark_multigenesets/multi_geneset.{}.{}.{}.{}.{}.txt'.format(name_of_dataset, es_metric, binary_or_cont, bench_date, i),header=None, index=False,sep='\t')
print(shuffled_es_df.head())

                 Bladder.bladder_cell  Bladder.bladder_urothelial_cell  \
gene                                                                     
ENSG00000081791                   1.0                              0.0   
ENSG00000162929                   0.0                              0.0   
ENSG00000168887                   0.0                              0.0   
ENSG00000162384                   1.0                              1.0   
ENSG00000154274                   0.0                              1.0   

                 Brain_Myeloid.macrophage  Brain_Myeloid.microglial_cell  \
gene                                                                       
ENSG00000081791                       0.0                            1.0   
ENSG00000162929                       0.0                            0.0   
ENSG00000168887                       0.0                            1.0   
ENSG00000162384                       0.0                            0.0   
ENSG00000154274          

# Adding random vector of noise to generate null 2

In [33]:
# Example
df = pd.DataFrame({'a':np.linspace(0,1,11), 'b':np.linspace(0,1,11)})
print(df)
noise = np.random.uniform(high=1, low=0, size=df.shape) 
print(df+noise)

      a    b
0   0.0  0.0
1   0.1  0.1
2   0.2  0.2
3   0.3  0.3
4   0.4  0.4
5   0.5  0.5
6   0.6  0.6
7   0.7  0.7
8   0.8  0.8
9   0.9  0.9
10  1.0  1.0
           a         b
0   0.748417  0.809980
1   0.117141  1.033725
2   0.850953  0.493446
3   0.874934  0.344119
4   0.695678  0.862065
5   0.984843  0.785851
6   0.727208  0.727941
7   1.352953  1.135058
8   0.863859  0.892788
9   1.737422  1.194014
10  1.324170  1.127896


In [34]:
es_metric = 'ESmu-NULL2'
binary_or_cont = 'continuous'
es_df = machine.metrics["cell_type.esmu"].copy()
# es_df.where(cond=es_df==0,other=1, inplace=True)
es_df.index.rename(name='gene',inplace=True)
print(es_df.head())
for i in range(5):
    noise = np.random.uniform(high=1, low=0, size=es_df.shape)
    noisy_es_df = es_df + noise
    noisy_es_df.reset_index(inplace=True)
    noisy_es_long_df = pd.melt(noisy_es_df,id_vars=['gene'],var_name='annotation', value_name='specificity')
    multi_geneset_celltypes = noisy_es_long_df[['annotation','gene','specificity']]
    multi_geneset_celltypes = multi_geneset_celltypes.loc[multi_geneset_celltypes.specificity>0]
    multi_geneset_celltypes.to_csv('../../data/benchmark_multigenesets/multi_geneset.{}.{}.{}.{}.{}.txt'.format(name_of_dataset, es_metric, binary_or_cont, bench_date, i),header=None, index=False,sep='\t')
print(noisy_es_df.head())

                 Bladder.bladder_cell  Bladder.bladder_urothelial_cell  \
gene                                                                     
ENSG00000081791               0.02842                         0.000000   
ENSG00000162929               0.00000                         0.000000   
ENSG00000168887               0.00000                         0.000000   
ENSG00000162384               0.34127                         0.175932   
ENSG00000154274               0.00000                         0.679072   

                 Brain_Myeloid.macrophage  Brain_Myeloid.microglial_cell  \
gene                                                                       
ENSG00000081791                  0.000000                       0.043131   
ENSG00000162929                  0.000000                       0.000000   
ENSG00000168887                  0.000000                       0.056487   
ENSG00000162384                  0.000000                       0.000000   
ENSG00000154274          

# CELLECT ESmu

In [8]:
es_metric = 'ESmu'
binary_or_cont = 'binary'

esmu_df = pd.read_csv('out_relevance/cell_type.esmu.190722.mapped.csv.gz',index_col=0)
# esmu_df = machine.metrics["cell_type.esmu"].copy()
esmu_df.where(cond=esmu_df==0,other=1, inplace=True)
esmu_df.index.rename(name='gene',inplace=True)
esmu_df.reset_index(inplace=True)
esmu_long_df = pd.melt(esmu_df,id_vars=['gene'],var_name='annotation', value_name='specificity')
multi_geneset_celltypes = esmu_long_df[['annotation','gene','specificity']]
multi_geneset_celltypes = multi_geneset_celltypes.loc[multi_geneset_celltypes.specificity>0]
multi_geneset_celltypes.to_csv('../../data/benchmark_multigenesets/multi_geneset.{}.{}.{}.{}.txt'.format(name_of_dataset, es_metric, binary_or_cont, bench_date),header=None, index=False,sep='\t')
multi_geneset_celltypes.head()

,annotation,gene,specificity
0,Bladder.bladder_cell,ENSG00000081791,1.0
3,Bladder.bladder_cell,ENSG00000162384,1.0
6,Bladder.bladder_cell,ENSG00000110696,1.0
10,Bladder.bladder_cell,ENSG00000137720,1.0
14,Bladder.bladder_cell,ENSG00000149179,1.0


In [36]:
es_metric = 'ESmu'
binary_or_cont = 'continuous'

esmu_df = machine.metrics["cell_type.esmu"].copy()
# esmu_df.where(cond=esmu_df==0,other=1, inplace=True)
esmu_df.index.rename(name='gene',inplace=True)
esmu_df.reset_index(inplace=True)
esmu_long_df = pd.melt(esmu_df,id_vars=['gene'],var_name='annotation', value_name='specificity')
multi_geneset_celltypes = esmu_long_df[['annotation','gene','specificity']]
multi_geneset_celltypes = multi_geneset_celltypes.loc[multi_geneset_celltypes.specificity>0]
multi_geneset_celltypes.to_csv('../../data/benchmark_multigenesets/multi_geneset.{}.{}.{}.{}.txt'.format(name_of_dataset, es_metric, binary_or_cont, bench_date),header=None, index=False,sep='\t')
multi_geneset_celltypes.head()

,annotation,gene,specificity
0,Bladder.bladder_cell,ENSG00000081791,0.028420
3,Bladder.bladder_cell,ENSG00000162384,0.341270
6,Bladder.bladder_cell,ENSG00000110696,0.031943
10,Bladder.bladder_cell,ENSG00000137720,0.046799
14,Bladder.bladder_cell,ENSG00000149179,0.218278


In [37]:
es_metric = 'ESmu'
binary_or_cont = 'continuous-squared'

esmu_df = machine.metrics["cell_type.esmu"].copy()
# esmu_df.where(cond=esmu_df==0,other=1, inplace=True)
esmu_df.index.rename(name='gene',inplace=True)
esmu_df = esmu_df**2
esmu_df.reset_index(inplace=True)
esmu_long_df = pd.melt(esmu_df,id_vars=['gene'],var_name='annotation', value_name='specificity')
multi_geneset_celltypes = esmu_long_df[['annotation','gene','specificity']]
multi_geneset_celltypes = multi_geneset_celltypes.loc[multi_geneset_celltypes.specificity>0]
multi_geneset_celltypes.to_csv('../../data/benchmark_multigenesets/multi_geneset.{}.{}.{}.{}.txt'.format(name_of_dataset, es_metric, binary_or_cont, bench_date),header=None, index=False,sep='\t')
multi_geneset_celltypes.head()

,annotation,gene,specificity
0,Bladder.bladder_cell,ENSG00000081791,0.000808
3,Bladder.bladder_cell,ENSG00000162384,0.116465
6,Bladder.bladder_cell,ENSG00000110696,0.001020
10,Bladder.bladder_cell,ENSG00000137720,0.002190
14,Bladder.bladder_cell,ENSG00000149179,0.047645
